In [1]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [3]:
local_path = "/Users/fajrzafar/Documents/Semester 8 /FYP 1/PDF FILES/output.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [4]:
# Preview first page
data[0].page_content

'Text\n\nClick https://bit.ly/FG-Books to download all PDF FG books for FREE Holiday Fun Pre-reading Look at the given picture and talk about the hilly areas. Read the highlighted words and tell their meanings. My name is Hafsa. I love to visit different places during summer holidays with my family. This time, we decided to go to Murree. First, I finished my homework. Then, I packed my luggage with my family. We had bags full of woollen clothes and jackets. We went to Murree by car. My grandparents also live there. I was so happy to see them. I had a great time with my grandparents. On the first day, we went to the Murree Hills. We While reading had a ride on the chairlift. It was an exciting Have you ever ridden experience. We also had a stroll on Mall Road. a chairlift? How was your The next day, we went to Nathia Gali. We drove experience? to the lush green mountain top. There were naughty monkeys all around. We had a view of the beautiful valley from the mountain top. It started ra

In [7]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pul

In [5]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED    
mistral:7b             	61e88e884507	4.1 GB	4 weeks ago	
nomic-embed-text:latest	0a109f422b47	274 MB	4 weeks ago	


In [10]:
!ollama pull mistral:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling e8a35b5937a5... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling e6836092461f... 100% ▕████████████████▏   42 B                         
pulling ed11eda7790d... 100% ▕████████████████▏   30 B                         
pulling f9b1e3196ecf... 100% ▕████████████████▏  483 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling e8a35b5937a5... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling e6836092461f... 100% ▕████████████████▏   42 B                         
pulling ed11eda7790d... 100% ▕██

In [6]:
%pip install --q chromadb
%pip install --q langchain-text-splitters


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [8]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [9]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)


OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00,  1.15it/s]


In [10]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [11]:
# LLM from Ollama
local_model = "mistral:7b"
llm = ChatOllama(model=local_model)

In [12]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [13]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [15]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [16]:
result = chain.invoke("What is this about?")


OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00,  1.17it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 48.23it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 57.05it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 57.47it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 52.41it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [17]:
print(result)

 This text is about a girl named Hafsa and her summer holiday experience at Murree with her family. They visited various hilly areas, went on chairlifts, had strolls, encountered monkeys, and enjoyed the natural beauty while hiking. The text also encourages correct pronunciation and intonation when reading.


In [18]:
from gtts import gTTS
import os

In [19]:
tts = gTTS(text=result, lang='en')
tts.save("/Users/fajrzafar/Documents/Semester 8 /FYP 1/generated_audio.mp3")

In [20]:
os.system("afplay generated_audio.mp3")


0